<a href="https://colab.research.google.com/github/EVA4-RS-Group/Phase2/blob/master/S13_SpeechRecognition/EVA4P2_Session13_Speech_recognition_model1_inference_v1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Loading the required libraries and dataset

In [1]:
!pip install torchaudio -q

     |████████████████████████████████| 7.6MB 12.6MB/s 


In [11]:
!rm -rf ./*
!wget https://github.com/EVA4-RS-Group/Phase2/releases/download/S13/sample_test_data.zip -q
!unzip -q sample_test_data.zip
!rm -rf /content/__MACOSX
!rm -rf /content/sample_test_data.zip
!wget -q https://github.com/EVA4-RS-Group/Phase2/releases/download/S13/weights_cpu_voicerec.pt

In [15]:
import glob
sample_file_list = list(glob.iglob('/content/sample_test_data/*.wav', recursive=True))

In [12]:
import torch
import torchaudio
import random 

/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


# 2. Define the network

In [13]:
class SpeechRNN(torch.nn.Module):
  
  def __init__(self):
    super(SpeechRNN, self).__init__()
    
    self.lstm = torch.nn.GRU(input_size = 12, 
                              hidden_size= 256, 
                              num_layers = 2, 
                              batch_first=True)
    
    self.out_layer = torch.nn.Linear(256, 30)
    
    self.softmax = torch.nn.LogSoftmax(dim=1)
    
  def forward(self, x):
    
    out, _ = self.lstm(x)
    
    x = self.out_layer(out[:,-1,:])
    
    return self.softmax(x)

classes = ['cat', 'dog', 'six', 'bird', 'eight', 'no', 'tree', 'marvin', 'left',
           'down', 'off', 'on', 'five', 'three', 'go', 'seven', 'sheila', 
           'right', 'four', 'happy', 'bed', 'zero', 'one', 'wow', 'two', 'yes',
           'house', 'up', 'nine', 'stop']

In [14]:
DEVICE=torch.device('cpu')
model = SpeechRNN()
model = model.to(DEVICE)

model.load_state_dict(torch.load('/content/weights_cpu_voicerec.pt', map_location=DEVICE))

<All keys matched successfully>

In [37]:

wav_file = random.choice(sample_file_list)
waveform,_ = torchaudio.load(wav_file, normalization=True)
      
# if the waveform is too short (less than 1 second) we pad it with zeroes
if waveform.shape[1] < 16000:
    waveform = F.pad(input=waveform, pad=(0, 16000 - waveform.shape[1]), mode='constant', value=0)
                     
mfcc_transform = torchaudio.transforms.MFCC(n_mfcc=12, log_mels=True)
mfcc = mfcc_transform(waveform).squeeze(0).transpose(0,1)
x = mfcc.unsqueeze(0)

model.eval()
y = model(x)
predicted_label = classes[y.max(1)[1].numpy().item()]

/usr/local/lib/python3.6/dist-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


In [38]:
print(f'Prediction of input file {wav_file.split("/")[-1]} is {predicted_label}.')

Prediction of input file marvin_20d3f11f_nohash_0.wav is marvin.
